# Parse the FIX message of a trade position 

to extract the position info from it

they start with `35=AP`

There are several, parsed with a while loop, from which we want to return an updated list of positions.

Maybe we build a new list from scratch and then replace the older list? In current code not clear if positions are also removed.

## Example

`55=14|710=63|721=339825679|727=10|728=0|730=1.59431|702=1|704=1000|705=0|10=215|`

## Relevant Fields


* 55 = asset
* 721 = position_id
* 727 = total number of positions opened now (this one is one of them)
* 728 = 0 valid response; 2 there is no open position
* 721 = 1 for a valid request
* 704 = quantity that is long; if short, this is 0
* 705 = quantity that is short; if this is long, this is zero
* 730 = settlement price; average cost price of the current position

These below not seen in our messages though:

* 1000 = Absolute TP
* 1002 = Absolute SL
* 1004 = Trailing SL
* 1005 = Trigger Method SL
* 1006 = Guaranteed SL

In [ ]:
import re

from assets import assets_all, DICT_SYMBOL_ID_SYMBOL

In [ ]:
DICT_SYMBOL_ID_SYMBOL

In [ ]:
messages = [
    "8=FIX.4.4|9=171|35=AP|34=2512|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=14|710=63|721=339825679|727=10|728=0|730=1.59431|702=1|704=1000|705=0|10=215|",
    "8=FIX.4.4|9=170|35=AP|34=2513|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=1|710=63|721=339825684|727=10|728=0|730=1.07468|702=1|704=1000|705=0|10=162|",
    "8=FIX.4.4|9=168|35=AP|34=2514|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=41|710=63|721=339825688|727=10|728=0|730=1960.48|702=1|704=1|705=0|10=084|",
    "8=FIX.4.4|9=169|35=AP|34=2515|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=10019|710=63|721=339825690|727=10|728=0|730=70.4|702=1|704=50|705=0|10=120|",
    "8=FIX.4.4|9=171|35=AP|34=2516|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=10012|710=63|721=339825699|727=10|728=0|730=1868.28|702=1|704=1|705=0|10=230|",
    "8=FIX.4.4|9=171|35=AP|34=2517|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=10015|710=63|721=339825701|727=10|728=0|730=33889.5|702=1|704=1|705=0|10=221|",
    "8=FIX.4.4|9=171|35=AP|34=2518|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=10014|710=63|721=339825713|727=10|728=0|730=14556.8|702=1|704=1|705=0|10=217|",
    "8=FIX.4.4|9=170|35=AP|34=2519|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=10013|710=63|721=339825725|727=10|728=0|730=4305.5|702=1|704=1|705=0|10=159|",
    "8=FIX.4.4|9=169|35=AP|34=2520|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=10015|710=63|721=339826567|727=10|728=0|730=33881|702=1|704=1|705=0|10=126|",
    "8=FIX.4.4|9=175|35=AP|34=2521|49=cServer|50=TRADE|52=20230610-20:00:43.780|56=demo.icmarkets.8739125|57=1McFMCBnn|55=10026|710=63|721=339826570|727=10|728=0|730=26439.83|702=1|704=0|705=0.01|10=160|",
]
messages

## Explaining the match

* to capture an integer: `re.search("721=(\\d+)", full_message)`: This statement searches for the pattern `721=` followed by one or more digits `(\d+)` within the `full_message` string. The parentheses around `\d+` capture the digits as a group, allowing you to access the matched value later.

* to capture a float we use the fact that it has `|` as a separator: `re.search("730=([\d.]+)\|"`

* `match := re.search(...)`: The walrus operator (`:=`) is used to assign the result of the search operation to the variable match. If a match is found, match will store a match object containing information about the match. If no match is found, match will be assigned `None`.

* `match` is of type `<class 're.Match'>` and has a value of `match=<re.Match object; span=(127, 140), match='721=339825679'>`

* `match.group(0)` returns all the string matched, so `721=339825679`

* `match.group(1)` returns the digits after the equal sign, so `339825679`

In [ ]:
def parse_one_position_message(full_message):
    d = None
    if match := re.search("721=(\\d+)", full_message):
        # print(type(match))
        # print(f"match={match}")
        # print(f"match.group(0)={match.group(0)}")
        # print(f"match.group(1)={match.group(1)}")
        d = {}
        d["position_id"] = match.group(1)
        #
        if match := re.search("55=(\\d+)", full_message):
            symbol_id = int(match.group(1))
        else:
            symbol_id = 0
        # print(f"symbold_id={symbol_id}")
        #
        if match := re.search("704=([\d.]+)\|", full_message):
            quantity_buy = float(match.group(1))
        else:
            quantity_buy = 0
        if match := re.search("705=([\d.]+)\|", full_message):
            quantity_sell = float(match.group(1))
        else:
            quantity_sell = 0.0
        if quantity_buy > 0.0 and quantity_sell == 0.0:
            direction = "buy"
            quantity = quantity_buy
        elif quantity_buy == 0.0 and quantity_sell > 0.0:
            direction = "sell"
            quantity = quantity_sell
        else:
            raise ValueError
        #
        if match := re.search("730=([\d.]+)\|", full_message):
            cost_price = float(match.group(1))
        else:
            cost_price = 0.0
        # 
        if match := re.search("727=(\\d+)", full_message):
            num_opened_positions = int(match.group(1))
        else:
            num_opened_positions = 0
        # build dictionary
        d["symbol"] = DICT_SYMBOL_ID_SYMBOL[symbol_id]
        d["symbol_id"] = symbol_id
        d["direction"] = direction
        d["quantity"] = quantity
        d["cost_price"] = cost_price
        d["num_opened_positions"] = num_opened_positions
        
    print(f"d={d}")
    return d




In [ ]:
positions = []
for message in messages:
    d = parse_one_position_message(message)
    if d is not None:
        positions.append(d)
positions

In [ ]:
try:
    positions = []
    for message in messages:
        d = parse_one_position_message(message)
        if d is not None:
            positions.append(d)
except:
    print(f"TRADE Unable to read from server")